In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configs

In [ ]:
LABEL2IDX = {
    'O': 0,
    'B-DAT': 1,
    'B-PER': 2,
    'B-ORG': 3,
    'B-LOC': 4,
    'B-EVE': 5,
    'I-DAT': 6,
    'I-PER': 7,
    'I-ORG': 8,
    'I-LOC': 9,
    'I-EVE': 10
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

CLS = [101]
SEP = [102]
VALUE_TOKEN = [0]
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 4
NUM_CLASS = 11
LEARNING_RATE = 5e-5

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast
# model_name = 'sbunlp/fabert'
model_name = 'PartAI/TookaBERT-Base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
train= "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/train.csv"
eval =  "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/eval.csv"
test =  "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/test.csv"

In [ ]:
def parse_tokens_and_labels(example):
    # Ensure 'tokens' and 'labels' are in the expected format
    if isinstance(example['tokens'], str):
        example['tokens'] = ast.literal_eval(example['tokens'])
    if isinstance(example['labels'], str):
        example['labels'] = ast.literal_eval(example['labels'])
    return example


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
dataset_train = load_dataset('csv', data_files=train)['train']
dataset_eval = load_dataset('csv', data_files=eval)['train']
dataset_test = load_dataset('csv', data_files=test)['train']

dataset_all = concatenate_datasets([dataset_train, dataset_eval, dataset_test])

# Apply the parsing function to the entire dataset
parsed_dataset = dataset_all.map(parse_tokens_and_labels)

# Rename labels to ner_tags
parsed_dataset = parsed_dataset.rename_column("labels", "ner_tags")

# Tokenize and align labels
tokenized_dataset = parsed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=parsed_dataset.column_names)

# Split the tokenized dataset back into train, eval, and test
dataset_length = len(dataset_train), len(dataset_eval), len(dataset_test)
print(dataset_length)



Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

(24000, 3000, 3000)


In [ ]:
combined = {
    'train': tokenized_dataset.select(range(dataset_length[0])),
    'eval': tokenized_dataset.select(range(dataset_length[0], dataset_length[0] + dataset_length[1])),
    'test': tokenized_dataset.select(range(dataset_length[0] + dataset_length[1], sum(dataset_length)))


In [ ]:
combined

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 24000
 }),
 'eval': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 3000
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 3000
 })}

In [ ]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {combined['train'][0]['input_ids']}, length: {len(combined['train'][0]['input_ids'])}")
print(f"Attention Mask: {combined['train'][0]['attention_mask']}, length: {len(combined['train'][0]['attention_mask'])}")
print(f"Labels: {combined['train'][0]['labels']}, length: {len(combined['train'][0]['labels'])}")

Input IDs: [2, 687, 802, 269, 121, 680, 689, 2807, 694, 3763, 802, 685, 4327, 4341, 6544, 4327, 1090, 5818, 1574, 1171, 862, 680, 1090, 3860, 1574, 1171, 862, 1851, 1574, 1171, 862, 4125, 1924, 680, 2308, 8501, 1552, 8925, 742, 169, 3], length: 41
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 41
Labels: [-100, 1, 1, 6, -100, 0, 0, -100, 0, 1, 6, 0, 0, 2, 7, 0, 3, 8, 8, 8, 8, 0, 3, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], length: 41


In [ ]:
 print(tokenizer.decode(combined['train'][0]['input_ids'][0]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][1]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][-2]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][-1]))

<s>
در
.
</s>


In [ ]:
print(tokenizer.convert_ids_to_tokens(combined['train'][0]['input_ids']))


['<s>', '▁در', '▁سال', '▁۱۳۸', '۵', '▁و', '▁ه', 'زمان', '▁با', '▁دومین', '▁سال', '▁ر', '▁رئیس\u200cجمهور', '▁محمود', '▁احمدی\u200cنژاد', '▁رئیس\u200cجمهور', '▁دولت', '▁نهم', '▁جمهوری', '▁اسلامی', '▁ایران', '▁و', '▁دولت', '▁دهم', '▁جمهوری', '▁اسلامی', '▁ایران', '▁نظام', '▁جمهوری', '▁اسلامی', '▁ایران', '▁معاونت', '▁علمی', '▁و', '▁فناوری', '▁رئیس\u200cجمهوری', '▁آغاز', '▁به\u200cکار', '▁کرد', '▁.', '</s>']


In [ ]:
train_dataset = combined['train']
eval_dataset = combined['eval']
test_dataset = combined['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24000
}) samples
Validation set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
}) samples
Test set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
}) samples


In [ ]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [ ]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [ ]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,    1,    1,    6, -100,    0,    0, -100,    0,    1,    6,    0,
            0,    2,    7,    0,    3,    8,    8,    8,    8,    0,    3,    8,
            8,    8,    8,    8,    8,    8,    8,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    3,    8,    0,    0,    0,    0,    0, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100]])

In [ ]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,    1,    1,    6, -100,    0,    0, -100,    0,    1,    6,    0,
           0,    2,    7,    0,    3,    8,    8,    8,    8,    0,    3,    8,
           8,    8,    8,    8,    8,    8,    8,    0,    0,    0,    0,    0,
           0,    0,    0,    0, -100])
tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    3,    8,    0,    0,    0,    0,    0, -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [ ]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d8dce7ab551ac4a8d0cbc521b1cad2c7a9e066a604d64396183e7385dd569336
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['O',
 'B-DAT',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'B-EVE',
 'I-DAT',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'I-EVE']

In [ ]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=NUM_CLASS, id2label=IDX2LABEL,label2id=LABEL2IDX)

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at PartAI/TookaBERT-Base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 41]),
 'attention_mask': torch.Size([1, 41]),
 'labels': torch.Size([1, 41])}

In [ ]:
 model.config.max_position_embeddings

512

In [ ]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})


In [45]:
training_args = TrainingArguments(
    output_dir = "/tmp/Persina-NER",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 10,
    weight_decay= 0,
    warmup_ratio = 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.220000,0.199310,0.580095,0.528886,0.553308,0.929749
2,0.178100,0.185118,0.620839,0.663688,0.641549,0.936412
3,0.121400,0.193961,0.606107,0.608535,0.607319,0.936465
4,0.080600,0.220264,0.641157,0.676321,0.658270,0.938371
5,0.057600,0.246078,0.638498,0.699584,0.667647,0.939028
6,0.039800,0.307922,0.628986,0.641196,0.635032,0.937871


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.220000,0.199310,0.580095,0.528886,0.553308,0.929749
2,0.178100,0.185118,0.620839,0.663688,0.641549,0.936412
3,0.121400,0.193961,0.606107,0.608535,0.607319,0.936465
4,0.080600,0.220264,0.641157,0.676321,0.658270,0.938371
5,0.057600,0.246078,0.638498,0.699584,0.667647,0.939028
6,0.039800,0.307922,0.628986,0.641196,0.635032,0.937871
7,0.023000,0.352139,0.618696,0.684178,0.649791,0.937030
8,0.017200,0.367018,0.619714,0.681867,0.649307,0.936334
9,0.009100,0.430544,0.632697,0.679094,0.655075,0.938791
10,0.006600,0.447680,0.632760,0.686181,0.658389,0.939278


TrainOutput(global_step=30000, training_loss=0.07864622463385264, metrics={'train_runtime': 4032.2371, 'train_samples_per_second': 59.52, 'train_steps_per_second': 7.44, 'total_flos': 7666004961985968.0, 'train_loss': 0.07864622463385264, 'epoch': 10.0})

In [46]:
model.config.num_labels

11

In [47]:
validation_results = trainer.evaluate()

In [ ]:
validation_results

{'eval_loss': 0.2264159917831421,
 'eval_precision': 0.6552529751040373,
 'eval_recall': 0.6917681516879913,
 'eval_f1': 0.673015634959319,
 'eval_accuracy': 0.941263011978867,
 'eval_runtime': 29.8538,
 'eval_samples_per_second': 200.98,
 'eval_steps_per_second': 25.122,
 'epoch': 4.0}

In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [49]:
test_results

{'eval_loss': 0.2451898455619812,
 'eval_precision': 0.6230681167716084,
 'eval_recall': 0.6857772877618522,
 'eval_f1': 0.6529204468771087,
 'eval_accuracy': 0.9391001933533175,
 'eval_runtime': 15.5857,
 'eval_samples_per_second': 192.484,
 'eval_steps_per_second': 24.06}

In [50]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         DAT     0.7404    0.8041    0.7709      2256

         EVE     0.4760    0.5000    0.4877       218

         LOC     0.5608    0.6334    0.5949      1863

         ORG     0.5419    0.5753    0.5581       876

         PER     0.5859    0.6576    0.6197      1136



   micro avg     0.6231    0.6858    0.6529      6349

   macro avg     0.5810    0.6341    0.6063      6349

weighted avg     0.6236    0.6858    0.6531      6349



In [51]:
model.push_to_hub("tookaBERT-base_finetuned")

model.safetensors:   0%|          | 0.00/489M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pouria82/tookaBERT-base_finetuned/commit/620982e0456e83dfcaa54d2f68d65bee14dffc8c', commit_message='Upload BertForTokenClassification', commit_description='', oid='620982e0456e83dfcaa54d2f68d65bee14dffc8c', pr_url=None, pr_revision=None, pr_num=None)